## Snowflake TPC-H & TPC-DS Data Import  
Import data from GCS to a previously created BigQuery dataset.  

This Notebook assumes that you've already generated data at one or more scale factors and uploaded them to the project Google Cloud Storage bucket listed in `config.gcs_data_bucket`  

Three values are required to initiate an upload to BigQuery:  
1. `test` - the test name, either `h` or `ds`
2. `scale` - the scale factor in GB, usually this will be `1, 100, 1000, 10000`  
3. `name` - name of this instance of the `test` and `scale` combination, i.e. `time-partitioned`

In [ ]:
import sfa, config

In [ ]:
test = "h"
scale = 100
cid = "01"
stream_n = 1
desc = "default"
seq_id = "d01"
verbose = True
verbose_query = True
qual = None
save = False
verbose_iter = True

In [ ]:
sf = sfa.SFTPC(test=test,
               scale=scale,
               cid=cid,
               warehouse="TEST9000",
               desc=desc,
               verbose=verbose,
               verbose_query=verbose_query)

In [ ]:
print('Using database:', sf.database)

In [ ]:
sf._connect()

In [ ]:
sf.role("SYSADMIN")

In [ ]:
sf.database_create()

In [ ]:
sf.create_schema({"ds": config.fp_sf_ds_schema,
                  "h": config.fp_sf_h_schema}[test])

In [ ]:
sf.gcs_integration_drop()
sf.gcs_integration_create()

In [ ]:
sf.grant_storage_integration_access()

In [ ]:
sf.gcs_inventory()

In [ ]:
sf.create_named_file_format()

In [ ]:
sf.warehouse_use()

In [ ]:
%%time
sf.import_data()